<h3>Group level sensor domain analysis</h3>

Imports

In [ ]:
import os
from os import path as op

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

import numpy as np

import mne
from mne import (read_evokeds, grand_average)
from mne.stats import permutation_t_test
from mne.viz import plot_evoked_topo

from bad_baby.picks import ses
mne.set_log_level(verbose='WARNING')
%matplotlib inline

Useful parameters

In [ ]:
lpf = 40
analysis = 'Deviants'
conditions = ['deviants']
studydir = '/media/ktavabi/ALAYA/data/ilabs/bad-baby/mismatch/'
fig_dir = op.join(studydir, 'figures')
if not op.isdir(fig_dir):
    os.mkdir(fig_dir)

### Field maps of grand averaged evoked data
Note here `mne.grand_average` assumes all eovoked datasets use an identical device to head transformation matrix, but since that is not the case the topological plots are distorted.

Average evoked data with `mne.grand_average`

In [ ]:
evokeds = []
subjects = ses['mismatch']['low']+ses['mismatch']['high']
for si, subj in enumerate(subjects):
    evoked_file = op.join(studydir, 'bad_%s' % subj, 'inverse',
                          '%s_%d-sss_eq_bad_%s-ave.fif' % (analysis, lpf,
                                                       subj))
    evoked = read_evokeds(evoked_file, condition=conditions[0], baseline=(None,0))
    if (evoked.info['sfreq']) != 600.0:
        print('bad_%s - %d' % (subj, evoked.info['sfreq']))
    evokeds.append(evoked.copy())
    evoked.pick_types(meg='grad', eeg=False, stim=False, eog=True,
                      include=[], exclude=[])
    if subj == subjects[0]:
        erf_data = np.zeros((len(subjects), len(evoked.info['chs']),
                            len(evoked.times)))
    erf_data[si] = evoked.data    

grndavr = grand_average(evokeds)
grndavr.save(op.join(studydir, '%s_%d_N%d_grand-ave.fif' % (analysis, lpf, len(subjects))))

In [ ]:
print('\nN=%d' % len(subjects))
hs = grndavr.plot_joint(ts_args={'gfp' : True}, topomap_args={'outlines' : 'head'})
for h, ch_type in zip(hs, ['grad', 'mag']):
    h.savefig(op.join(studydir, fig_dir, '%s_%d_%s_N%d-ave.png'
                      % (conditions[0], lpf, ch_type, len(subjects))), dpi=240, format='png')

### Sesnsor permutation t-test 
Test whether ERF signal across subjects significantly deviates from 0 using sensor permutation to map significantly responsive sensors in the interval 200-300ms post-stimulus. 

In [ ]:
# Apply temporal mask to select time window
times = grndavr.times
temporal_mask = np.logical_and(.2 <= times, times <= .3)
grndavr.pick_types(meg='grad', exclude=[])
picks = picks = mne.pick_types(grndavr.info, meg='grad', eeg=False, stim=False, eog=False, 
                               include=[], exclude=[])
data = np.mean(erf_data[:, :, temporal_mask], axis=2)
# T-test
T0, p_values, H0 = permutation_t_test(data, n_permutations=1000, n_jobs=18)
# find significant sensors
significant_sensors = picks[p_values <= .05]
print("Number of significant sensors : %d" % len(significant_sensors))
print("Sensors names : MEG%s" % significant_sensors)
evoked = mne.EvokedArray(-np.log10(p_values)[:, np.newaxis],
                         grndavr.info, tmin=0.)
stats_picks = mne.pick_channels(grndavr.ch_names, significant_sensors)
mask = p_values[:, np.newaxis] <= 0.05
fig = evoked.plot_topomap(ch_type='grad', times=[0], scale=1, 
                          time_format=None, cmap='Reds', vmin=0., vmax=np.max,
                          unit='-log10(p)', cbar_fmt='-%0.1f', mask=mask,
                          size=3, show_names=lambda x: x[0:] + ' ' * 20)
fig.savefig(op.join(fig_dir, '%s_%d_N%d_200-300ms_sensorSPM.png' % (analysis,
                                                                   lpf,
                                                                   len(subjects))))

Plot histogram for group ERF data and observed t-values from permutation testing.

In [ ]:
n, bins, patches = plt.hist(np.ravel(erf_data), 100, normed=1, facecolor='green', alpha=0.75)
mean = np.mean(erf_data)
std = np.std(erf_data)
# add a 'best fit' line
y = mlab.normpdf( bins, mean, std)
l = plt.plot(bins, y, 'r--', linewidth=1)
plt.xlabel('Amplitude')
plt.ylabel('Probability')
plt.grid(True)
plt.title(r'$ERF\ data:\ M=%0.2f,\ STD=%0.2f$' % (mean, 
                                                                 std))
plt.show()

n, bins, patches = plt.hist(np.ravel(T0), 100, normed=1, facecolor='green', alpha=0.75)
plt.xlabel('T values')
plt.ylabel('Probability')
plt.grid(True)
plt.title(r'$Biased\ T-Values$')
plt.show()

### Proof of concept
Copy ERF data and bias single sensor and plot data histogram. Next, re-run permutation sensor t-test and plot observed t-values examining effect of biased sensor.

In [ ]:
arr = np.copy(erf_data)
arr[:,[11],:]+=100e-13
print(np.where(erf_data < arr))  # shows biased channel

n, bins, patches = plt.hist(np.ravel(arr), 100, normed=1, facecolor='green', alpha=0.75)
mean = np.mean(erf_data)
std = np.std(erf_data)
# add a 'best fit' line
y = mlab.normpdf( bins, mean, std)
l = plt.plot(bins, y, 'r--', linewidth=1)
plt.xlabel('Amplitudes')
plt.ylabel('Probability')
plt.grid(True)
plt.title(r'$Biased\ ERF\ data:\ M=%0.2f,\ STD=%0.2f$' % (mean, 
                                                                 std))
plt.show()

In [ ]:
data = np.mean(arr[:, :, temporal_mask], axis=2)
# T-test
T0, p_values, H0 = permutation_t_test(data, n_permutations=1000, n_jobs=18)
# find significant sensors
significant_sensors = picks[p_values <= .05]
print("Number of significant sensors : %d" % len(significant_sensors))
print("Sensors names : MEG%s" % significant_sensors)
evoked = mne.EvokedArray(-np.log10(p_values)[:, np.newaxis],
                         grndavr.info, tmin=0.)
stats_picks = mne.pick_channels(evoked.ch_names, significant_sensors)
mask = p_values[:, np.newaxis] <= 0.05
fig = evoked.plot_topomap(ch_type='grad', times=[0], scale=1, 
                          time_format=None, cmap='Reds', vmin=0., vmax=np.max,
                          unit='-log10(p)', cbar_fmt='-%0.1f', mask=mask,
                          size=3, show_names=lambda x: x[0:] + ' ' * 20)

n, bins, patches = plt.hist(np.ravel(T0), 100, normed=1, facecolor='green', alpha=0.75)
plt.xlabel('T values')
plt.ylabel('Probability')
plt.title(r'$Biased\ T-Values$')
plt.grid(True)
plt.show()

Test whether ERF signal across subjects significantly deviates from 0 using Bonferroni adjustment to map significantly response sensors in the interval 200-300ms post-stimulus onset.

In [ ]:
from scipy.stats import ttest_1samp as ttest

data = np.mean(erf_data[:, :, temporal_mask], axis=2)
T0, p_values = ttest(data, 0, 0)
# Bonferroni adjust
d = np.sign(T0) * -np.log10(np.minimum(np.abs(p_values) * len(grndavr.info['chs']), 1))  # signed log10(p) 

n, bins, patches = plt.hist(d, 100, normed=1, facecolor='green', alpha=0.75)
plt.xlabel('log10(p)')
plt.ylabel('Probability')
plt.title(r'$log10(p-values)$')
plt.grid(True)
plt.show()

# find significant sensors
significant_sensors = picks[np.abs(d) >= 1.3]
print("Number of significant sensors : %d" % len(significant_sensors))
print("Sensors names : MEG%s" % significant_sensors)
evoked = mne.EvokedArray(-d[:, np.newaxis],
                         grndavr.info, tmin=0.)
stats_picks = mne.pick_channels(evoked.ch_names, significant_sensors)
mask = np.abs(d[:, np.newaxis]) >= 1.3
fig = evoked.plot_topomap(ch_type='grad', times=[0], scale=1, 
                          time_format=None, cmap='Reds', vmin=0., vmax=np.max,
                          unit='-log10(p)', cbar_fmt='-%0.1f', mask=mask,
                          size=3, show_names=lambda x: x[0:] + ' ' * 20)

### Sesnsor permutation t-test 
For each individual ERF data test if the signal significantly deviates from 0 in the interval 200-300ms post-stimulus onset, i.e., reliability of evoked response.

## Groups
### Low SES

In [ ]:
groups = {'low': mismatch_subjects['low_ses'],
          'high': mismatch_subjects['high_ses']}

In [ ]:
evokeds = []
subjects = groups['low']
for si, subj in enumerate(subjects):
    evoked_file = op.join(studydir, 'bad_%s' % subj, 'inverse',
                          '%s_%d-sss_eq_bad_%s-ave.fif' % (analysis, lpf,
                                                       subj))
    evoked = read_evokeds(evoked_file, condition=conditions[0], baseline=(None,0))
    if (evoked.info['sfreq']) != 600.0:
        print('bad_%s - %d' % (subj, evoked.info['sfreq']))
    evokeds.append(evoked.copy())
    evoked.pick_types(meg='grad', eeg=False, stim=False, eog=True,
                      include=[], exclude=[])
    if subj == subjects[0]:
        erf_data = np.zeros((len(subjects), len(evoked.info['chs']),
                            len(evoked.times)))
    erf_data[si] = evoked.data    

grndavr_lowses = grand_average(evokeds)
grndavr_lowses = grand_average(evokeds)
grndavr_lowses.save(op.join(studydir,'%s_%d_N%d_lowses_grand-ave.fif' % (analysis, lpf, 
                                                                     len(subjects))))

In [ ]:
print('\nn=%d' % len(groups['low']))
ylim = {'mag': [-25, 25], 'grad': [-80, 80]}
hs = grndavr_lowses.plot_joint(ts_args={'gfp' : True}, topomap_args={'outlines' : 'head'})
for h, ch_type in zip(hs, ['grad', 'mag']):
    h.savefig(op.join(studydir, fig_dir, '%s_%d_%s_lowses-N%d-ave.png'
                      % (conditions[0], lpf, ch_type, len(groups['low']))), dpi=240, format='png')

In [ ]:
# Apply temporal mask to select time window
times = grndavr_lowses.times
temporal_mask = np.logical_and(.2 <= times, times <= .3)
grndavr_lowses.pick_types(meg='grad', exclude=[])
picks = picks = mne.pick_types(grndavr_lowses.info, meg='grad', eeg=False, stim=False, eog=False, 
                               include=[], exclude=[])
assert erf_data.shape[0] == len(subjects)
data = np.mean(erf_data[:, :, temporal_mask], axis=2)
# T-test
T0, p_values, H0 = permutation_t_test(data, n_permutations=1000, n_jobs=18)
# find significant sensors
significant_sensors = picks[p_values <= .05]
print("Number of significant sensors : %d" % len(significant_sensors))
print("Sensors names : MEG%s" % significant_sensors)
evoked = mne.EvokedArray(-np.log10(p_values)[:, np.newaxis],
                         grndavr_lowses.info, tmin=0.)
stats_picks = mne.pick_channels(grndavr_lowses.ch_names, significant_sensors)
mask = p_values[:, np.newaxis] <= 0.05
fig = evoked.plot_topomap(ch_type='grad', times=[0], scale=1, 
                          time_format=None, cmap='Reds', vmin=0., vmax=np.max,
                          unit='-log10(p)', cbar_fmt='-%0.1f', mask=mask,
                          size=3, show_names=lambda x: x[0:] + ' ' * 20)
fig.savefig(op.join(fig_dir, '%s_%d_N%d_lowses_200-300ms_sensorSPM.png' % (analysis,
                                                                   lpf,
                                                                   len(subjects))))
signif_chs = {'lowses': [], 'highses': []}
signif_chs['lowses'] = [grndavr_lowses.info['ch_names'][i] for i in significant_sensors]

### High SES

In [ ]:
evokeds = []
subjects = groups['high']
for si, subj in enumerate(subjects):
    evoked_file = op.join(studydir, 'bad_%s' % subj, 'inverse',
                          '%s_%d-sss_eq_bad_%s-ave.fif' % (analysis, lpf,
                                                       subj))
    evoked = read_evokeds(evoked_file, condition=conditions[0], baseline=(None,0))
    if (evoked.info['sfreq']) != 600.0:
        print('bad_%s - %d' % (subj, evoked.info['sfreq']))
    evokeds.append(evoked.copy())
    evoked.pick_types(meg='grad', eeg=False, stim=False, eog=True,
                      include=[], exclude=[])
    if subj == subjects[0]:
        erf_data = np.zeros((len(subjects), len(evoked.info['chs']),
                            len(evoked.times)))
    erf_data[si] = evoked.data    

grndavr_highses = grand_average(evokeds)
grndavr_highses = grand_average(evokeds)
grndavr_highses.save(op.join(studydir,'%s_%d_N%d_highses_grand-ave.fif' % (analysis, lpf, 
                                                                     len(subjects))))
signif_chs['highses'] = [grndavr_highses.info['ch_names'][i] for i in significant_sensors]

In [ ]:
print('\nn=%d' % len(groups['high']))
ylim = {'mag': [-25, 25], 'grad': [-80, 80]}
hs = grndavr_highses.plot_joint(ts_args={'gfp' : True}, topomap_args={'outlines' : 'head'})
for h, ch_type in zip(hs, ['grad', 'mag']):
    h.savefig(op.join(studydir, fig_dir, '%s_%d_%s_highses-N%d-ave.png'
                      % (conditions[0], lpf, ch_type, len(groups['low']))), dpi=240, format='png')

In [ ]:
# Apply temporal mask to select time window
times = grndavr_highses.times
temporal_mask = np.logical_and(.2 <= times, times <= .3)
grndavr_highses.pick_types(meg='grad', exclude=[])
picks = picks = mne.pick_types(grndavr_highses.info, meg='grad', eeg=False, stim=False, eog=False, 
                               include=[], exclude=[])
assert erf_data.shape[0] == len(subjects)
data = np.mean(erf_data[:, :, temporal_mask], axis=2)
# T-test
T0, p_values, H0 = permutation_t_test(data, n_permutations=1000, n_jobs=18)
# find significant sensors
significant_sensors = picks[p_values <= .05]
print("Number of significant sensors : %d" % len(significant_sensors))
print("Sensors names : MEG%s" % significant_sensors)
evoked = mne.EvokedArray(-np.log10(p_values)[:, np.newaxis],
                         grndavr_highses.info, tmin=0.)
stats_picks = mne.pick_channels(grndavr_highses.ch_names, significant_sensors)
mask = p_values[:, np.newaxis] <= 0.05
fig = evoked.plot_topomap(ch_type='grad', times=[0], scale=1, 
                          time_format=None, cmap='Reds', vmin=0., vmax=np.max,
                          unit='-log10(p)', cbar_fmt='-%0.1f', mask=mask,
                          size=3, show_names=lambda x: x[0:] + ' ' * 20)
fig.savefig(op.join(fig_dir, '%s_%d_N%d_highses_200-300ms_sensorSPM.png' % (analysis,
                                                                   lpf,
                                                                   len(subjects))))
signif_chs['highses'] = [grndavr_highses.info['ch_names'][i] for i in significant_sensors]

In [ ]:
evokeds = []
subjects = ses['mismatch']['young']
for si, subj in enumerate(subjects):
    evoked_file = op.join(studydir, 'bad_%s' % subj, 'inverse',
                          '%s_%d-sss_eq_bad_%s-ave.fif' % (analysis, lpf,
                                                       subj))
    evoked = read_evokeds(evoked_file, condition=conditions[0], baseline=(None,0))
    if (evoked.info['sfreq']) != 600.0:
        print('bad_%s - %d' % (subj, evoked.info['sfreq']))
    evokeds.append(evoked.copy())
    evoked.pick_types(meg='grad', eeg=False, stim=False, eog=True,
                      include=[], exclude=[])
    if subj == subjects[0]:
        erf_data = np.zeros((len(subjects), len(evoked.info['chs']),
                            len(evoked.times)))
    erf_data[si] = evoked.data    

grndavr = grand_average(evokeds)
#grndavr.save(op.join(studydir, '%s_%d_N%d_grand-ave.fif' % (analysis, lpf, len(subjects))))

In [ ]:
print('\nN=%d' % len(subjects))
hs = grndavr.plot_joint(ts_args={'gfp' : True}, topomap_args={'outlines' : 'head'})
for h, ch_type in zip(hs, ['grad', 'mag']):
    h.savefig(op.join(studydir, fig_dir, '%s_%d_%s_N%d_two-four_mns-ave.png'
                      % (conditions[0], lpf, ch_type, len(subjects))), dpi=240, format='png')

In [ ]:
# Apply temporal mask to select time window
times = grndavr.times
temporal_mask = np.logical_and(.2 <= times, times <= .3)
grndavr.pick_types(meg='grad', exclude=[])
picks = picks = mne.pick_types(grndavr.info, meg='grad', eeg=False, stim=False, eog=False, 
                               include=[], exclude=[])
data = np.mean(erf_data[:, :, temporal_mask], axis=2)
# T-test
T0, p_values, H0 = permutation_t_test(data, n_permutations=1000, n_jobs=18)
# find significant sensors
significant_sensors = picks[p_values <= .05]
print("Number of significant sensors : %d" % len(significant_sensors))
print("Sensors names : MEG%s" % significant_sensors)
evoked = mne.EvokedArray(-np.log10(p_values)[:, np.newaxis],
                         grndavr.info, tmin=0.)
stats_picks = mne.pick_channels(grndavr.ch_names, significant_sensors)
mask = p_values[:, np.newaxis] <= 0.05
fig = evoked.plot_topomap(ch_type='grad', times=[0], scale=1, 
                          time_format=None, cmap='Reds', vmin=0., vmax=np.max,
                          unit='-log10(p)', cbar_fmt='-%0.1f', mask=mask,
                          size=3, show_names=lambda x: x[0:] + ' ' * 20)
fig.savefig(op.join(fig_dir, '%s_%d_N%d_200-300ms_two-four_mns_sensorSPM.png' % (analysis,
                                                                                 lpf,
                                                                                 len(subjects))))

In [ ]:
evokeds = []
subjects = ses['mismatch']['old']
for si, subj in enumerate(subjects):
    evoked_file = op.join(studydir, 'bad_%s' % subj, 'inverse',
                          '%s_%d-sss_eq_bad_%s-ave.fif' % (analysis, lpf,
                                                       subj))
    evoked = read_evokeds(evoked_file, condition=conditions[0], baseline=(None,0))
    if (evoked.info['sfreq']) != 600.0:
        print('bad_%s - %d' % (subj, evoked.info['sfreq']))
    evokeds.append(evoked.copy())
    evoked.pick_types(meg='grad', eeg=False, stim=False, eog=True,
                      include=[], exclude=[])
    if subj == subjects[0]:
        erf_data = np.zeros((len(subjects), len(evoked.info['chs']),
                            len(evoked.times)))
    erf_data[si] = evoked.data    

grndavr = grand_average(evokeds)
#grndavr.save(op.join(studydir, '%s_%d_N%d_grand-ave.fif' % (analysis, lpf, len(subjects))))

In [ ]:
print('\nN=%d' % len(subjects))
hs = grndavr.plot_joint(ts_args={'gfp' : True}, topomap_args={'outlines' : 'head'})
for h, ch_type in zip(hs, ['grad', 'mag']):
    h.savefig(op.join(studydir, fig_dir, '%s_%d_%s_N%d_four-six_mns-ave.png'
                      % (conditions[0], lpf, ch_type, len(subjects))), dpi=240, format='png')

In [ ]:
# Apply temporal mask to select time window
times = grndavr.times
temporal_mask = np.logical_and(.2 <= times, times <= .3)
grndavr.pick_types(meg='grad', exclude=[])
picks = picks = mne.pick_types(grndavr.info, meg='grad', eeg=False, stim=False, eog=False, 
                               include=[], exclude=[])
data = np.mean(erf_data[:, :, temporal_mask], axis=2)
# T-test
T0, p_values, H0 = permutation_t_test(data, n_permutations=1000, n_jobs=18)
# find significant sensors
significant_sensors = picks[p_values <= .05]
print("Number of significant sensors : %d" % len(significant_sensors))
print("Sensors names : MEG%s" % significant_sensors)
evoked = mne.EvokedArray(-np.log10(p_values)[:, np.newaxis],
                         grndavr.info, tmin=0.)
stats_picks = mne.pick_channels(grndavr.ch_names, significant_sensors)
mask = p_values[:, np.newaxis] <= 0.05
fig = evoked.plot_topomap(ch_type='grad', times=[0], scale=1, 
                          time_format=None, cmap='Reds', vmin=0., vmax=np.max,
                          unit='-log10(p)', cbar_fmt='-%0.1f', mask=mask,
                          size=3, show_names=lambda x: x[0:] + ' ' * 20)
fig.savefig(op.join(fig_dir, '%s_%d_N%d_200-300ms_four-six_mns_sensorSPM.png' % (analysis,
                                                                                 lpf,
                                                                                 len(subjects))))